# Model Definition 4

Now that we've one-hot encoded the route data, there is no need for the hybrid model, so we'll use a simple 1D CNN instead.

Let's see how it does!

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

import plaidml.keras as pk
pk.install_backend()

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Conv1D, Dropout, LeakyReLU, MaxPooling1D, Embedding, Flatten, Input, Concatenate

### Reading in training and validation data

In [2]:
with open('./big_dummy_data.pickle', 'rb') as f:
    dataset = pickle.load(f)

In [3]:
X, y = dataset
print(X.shape)
print(y.shape)

(68486, 420)
(68486,)


In [40]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.33)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

### DeepLearning Methods:

### CNN

In [41]:
def build_model():

    inp = Input(shape=(420,1))
    
    x = Conv1D(1024, kernel_size=4, strides=1)(inp)
    x = LeakyReLU()(x)
    x = MaxPooling1D(pool_size=4)(x)
    
    x = Conv1D(512, kernel_size=4, strides=1)(inp)
    x = LeakyReLU()(x)
    x = MaxPooling1D(pool_size=4)(x)
    
    x = Conv1D(64, kernel_size=4, strides=1)(inp)
    x = LeakyReLU()(x)
    x = MaxPooling1D(pool_size=4)(x)
    
    x = Flatten()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inp, outputs=out)
    
    model.compile(loss='binary_crossentropy', optimizer='adam',
                 metrics=['acc'])
    
    return model

In [42]:
model = build_model()

In [43]:
learning_rate_reduction_combined = ReduceLROnPlateau(monitor='val_acc', patience=3, 
                                            verbose=2, factor=0.5, min_lr=0.00001)

best_model_combined = ModelCheckpoint('./simple_cnn.1.h5', monitor='val_acc', verbose=2, 
                             save_best_only=True, mode='max')

early_stopping_combined = EarlyStopping(monitor='val_loss', min_delta=1e-10, 
                               patience=10, restore_best_weights=True)

In [44]:
hist = model.fit(x_train, y_train,
         batch_size=32,
         epochs=50,
         validation_data=(x_test, y_test),
         callbacks = [learning_rate_reduction_combined, best_model_combined, early_stopping_combined],
         verbose=1
)

Train on 45885 samples, validate on 22601 samples
Epoch 1/50
45885/45885 [==============================] - 12s 270us/step - loss: 0.3188 - acc: 0.8758 - val_loss: 0.2817 - val_acc: 0.8958

Epoch 00001: val_acc improved from -inf to 0.89580, saving model to ./simple_cnn.1.h5
Epoch 2/50
45885/45885 [==============================] - 10s 219us/step - loss: 0.2610 - acc: 0.9060 - val_loss: 0.2620 - val_acc: 0.9074

Epoch 00002: val_acc improved from 0.89580 to 0.90739, saving model to ./simple_cnn.1.h5
Epoch 3/50
45885/45885 [==============================] - 10s 218us/step - loss: 0.2433 - acc: 0.9131 - val_loss: 0.2484 - val_acc: 0.9123

Epoch 00003: val_acc improved from 0.90739 to 0.91230, saving model to ./simple_cnn.1.h5
Epoch 4/50
45885/45885 [==============================] - 10s 218us/step - loss: 0.2325 - acc: 0.9170 - val_loss: 0.2383 - val_acc: 0.9157

Epoch 00004: val_acc improved from 0.91230 to 0.91571, saving model to ./simple_cnn.1.h5
Epoch 5/50
45885/45885 [=============

This is our best model yet, scoring 92.2% accuracy (v. the previous high of 91.7%). This is a fine example of how sometimes bigger doesn't mean better.

Let's evaluate our models in the model_evaluate.4 notebook.